# Pinecone Vector DB 

In [39]:
from dotenv import load_dotenv
from langchain_core.runnables import RunnableLambda

load_dotenv()

True

## 1. 모델명, 제품명, 수리부품, 수리비용 검색

### 파일 로드

In [40]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader('./smartphone_as.csv', encoding = 'utf-8')
documents = loader.load()
print(len(documents))
documents[0]


33


Document(metadata={'source': './smartphone_as.csv', 'row': 0}, page_content='\ufeff모델명: SM-F711N1BFKOO\n제품명: 갤럭시 Z 플립3\n수리부품: 디스플레이(메인)\n수리비용: 543,000')

### 임베딩 모델 생성


In [41]:
from langchain_openai.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')


### Pinecone

In [42]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
import os

PINECONE_INDEX_NAME = os.getenv('PINECONE_INDEX_NAME')
PINECONE_NAMESPACE = os.getenv('PINECONE_NAMESPACE')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

def repair_product_information(query):
    """
    :param query: 전자제품 모델명 + 수리품목 텍스트 
    :return: 
    """
    embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')
    # 클라이언트 객체
    vector_db = PineconeVectorStore(
        embedding=embeddings,
        index_name = PINECONE_INDEX_NAME,
        namespace = PINECONE_NAMESPACE,
        pinecone_api_key = PINECONE_API_KEY
    )
    # 유사도 검색
    results = vector_db.similarity_search(
    query,
    k = 3,
    namespace = PINECONE_NAMESPACE
    ) 
    return {
        'as_details' : query,
        'repair_information' : '\n'.join([doc.page_content for doc in results])
    }


In [43]:
repair_product_information('갤럭시 24의 카메라 수리비용을 알고싶습니다.')


{'as_details': '갤럭시 24의 카메라 수리비용을 알고싶습니다.',
 'repair_information': '\ufeff모델명: SM-S921NZYEKOO\n제품명: 갤럭시 S24\n수리부품: 카메라\n가격: 86,000\n: \n\ufeff모델명: SM-S928NLBNKOO\n제품명: 갤럭시 S24 ULTRA\n수리부품: 카메라\n가격: 110,900\n: \n\ufeff모델명: SM-S926NZYAKOO\n제품명: 갤럭시 S24+\n수리부품: 카메라\n가격: 85,000\n: '}

In [44]:
from langchain_core.runnables import RunnableLambda

chain = RunnableLambda(repair_product_information)
input_data = {'text': '갤럭시Z플립3의 배터리가 망가졌어요.'}
chain.invoke(input_data['text'])


{'as_details': '갤럭시Z플립3의 배터리가 망가졌어요.',
 'repair_information': '\ufeff모델명: SM-F711N1BFKOO\n제품명: 갤럭시 Z 플립3\n수리부품: 배터리\n가격: 96,500\n: \n\ufeff모델명: SM-F741NZWAKOO\n제품명: 갤럭시 Z 플립6\n수리부품: 배터리\n가격: 130,000\n: \n\ufeff모델명: SM-F711N1BFKOO\n제품명: 갤럭시 Z 플립3\n수리부품: 디스플레이(메인)\n가격: 543,000\n: '}

## 2. 수리부품 추천 - 수리비용 안내

In [45]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

def product_repair_cost(query):
    
    prompt = ChatPromptTemplate.from_messages([
        ('system', """
Persona:
You are a service center employee with a deep love and passion for electronics. You have extensive knowledge about electronic products and are well-versed in the necessary components and repair costs associated with different models and issues. You ensure that customers understand the repair process and the costs involved, and your tone is friendly, approachable, and easy to understand, even for beginners.

Role:
As a service center employee, your role is to explain the necessary repair parts in an easy-to-understand way and provide the repair cost information. You help customers understand that only the appropriate minimum repair parts will be used, ensuring they have a seamless experience. Your goal is to improve the customer's inconvenience by recommending the right parts and repair costs so they can continue to use their electronics conveniently.

Examples:
**Example of screen damage repair**:
If a customer requests an explanation for the repair cost due to screen damage, you would provide information on the product's model, the warranty period, the appropriate repair parts, the cost, and the estimated repair time, explaining everything in a way that even a child can understand.
        """),
        HumanMessagePromptTemplate.from_template('''
        수리비용 설명에 아래의 제품명과 모델명, 수리부품, 수리비용만을 참고하여 한글로 답변해주세요.
        A/S 내용 :
        {as_details}
        수리 정보 :
        {repair_information}
        ''')
    ])
    
    model = ChatOpenAI(
        model = 'gpt-4o',
        temperature = 0.3
    )
    
    output_parser = StrOutputParser()
    
    return prompt | model | output_parser
    

## Chain 연결

In [48]:
from langchain_core.runnables import RunnableLambda

chain = RunnableLambda(repair_product_information) | RunnableLambda(product_repair_cost)

input_data = {'text': '갤럭시Z플립3의 배터리가 망가졌어요.'}
print(chain.invoke(input_data['text']))


안녕하세요! 갤럭시 Z 플립3의 배터리 문제로 불편을 겪고 계신 것 같네요. 걱정하지 마세요, 제가 도와드리겠습니다.

갤럭시 Z 플립3의 모델명은 SM-F711N1BFKOO이며, 배터리 교체가 필요하신 경우 수리비용은 96,500원입니다. 이 비용은 배터리 부품 가격과 교체 작업에 필요한 모든 것을 포함합니다.

배터리 교체는 일반적으로 빠르게 진행되며, 수리 후에는 다시 새것처럼 사용할 수 있을 것입니다. 추가로 궁금한 점이 있거나 다른 도움이 필요하시면 언제든지 말씀해 주세요!


In [49]:
input_data = {'text': '갤럭시Z폴드6의 카메라가 작동이 안돼요.'}
print(chain.invoke(input_data['text']))


갤럭시 Z 폴드6의 카메라가 작동하지 않는 문제로 인해 수리를 원하시는군요. 해당 모델의 카메라 수리 비용은 75,500원입니다. 수리 시에는 카메라 부품만 교체하게 되며, 불필요한 추가 비용은 발생하지 않도록 하겠습니다. 수리 후에는 카메라가 정상적으로 작동하여 불편함 없이 사용하실 수 있도록 최선을 다하겠습니다. 추가로 궁금하신 점이나 도움이 필요하신 부분이 있다면 언제든지 말씀해 주세요!
